# Psychoinformatics - Week 11 (Exercises)
張天瑋

In [1]:
%config IPCompleter.greedy=True 
%matplotlib inline

from numpy import *
from matplotlib.pyplot import *
from IPython.display import *
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
from sklearn import *

iris = datasets.load_iris()
X=iris.data; Y=iris.target

## 1 Performance Tuning of a Neural Net (8 points)

### 1.0 Baseline Performance
SVM can reach an classifcation accuracy ~ 8x% correct for the hard Iris problem.

In [2]:
sss=model_selection.StratifiedShuffleSplit(n_splits=5,test_size=0.1) # (45+45+45) vs. (5+5+5)
model=svm.SVC(C=10)
acc=[]
for train_index, test_index in sss.split(X, Y): # 5-fold cross-validation
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    model.fit(X_train[:,0:2],Y_train) #training
    acc.append(model.predict(X_test[:,0:2])==Y_test) # testing
print(np.mean(acc))

0.7333333333333333


### 1.1 Tuning your ANN (4 points)
Tune your model hyperparameters (# of layers, # of units in each layer, activation function, optimizer, epochs, batch_size, etc.) to see if you can push your ANN performance up to ~9x% correct.

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.layers import Dropout

model = Sequential()
model.add(Dense(units=16, input_dim=4, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=8, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=4, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=3, kernel_initializer='uniform', activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

acc=[]
for train_index, test_index in sss.split(X, Y): # 5-fold cross-validation
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    model.fit(X_train[:,0:4], to_categorical(Y_train), epochs=200, batch_size=30)
    acc.append(argmax(model.predict(X_test[:,0:4]),1)==Y_test) # testing
print(np.mean(acc))

Epoch 1/200
5/5 [==============================] - 1s 4ms/step - loss: 1.0987 - accuracy: 0.2889
Epoch 2/200
5/5 [==============================] - 0s 5ms/step - loss: 1.0987 - accuracy: 0.3333
Epoch 3/200
5/5 [==============================] - 0s 4ms/step - loss: 1.0986 - accuracy: 0.3333
Epoch 4/200
5/5 [==============================] - 0s 4ms/step - loss: 1.0987 - accuracy: 0.2741
Epoch 5/200
5/5 [==============================] - 0s 4ms/step - loss: 1.0986 - accuracy: 0.3778
Epoch 6/200
5/5 [==============================] - 0s 4ms/step - loss: 1.0986 - accuracy: 0.3333
Epoch 7/200
5/5 [==============================] - 0s 4ms/step - loss: 1.0985 - accuracy: 0.3333
Epoch 8/200
5/5 [==============================] - 0s 5ms/step - loss: 1.0985 - accuracy: 0.3333
Epoch 9/200
5/5 [==============================] - 0s 6ms/step - loss: 1.0982 - accuracy: 0.3333
Epoch 10/200
5/5 [==============================] - 0s 4ms/step - loss: 1.0980 - accuracy: 0.3333
Epoch 11/200
5/5 [===========

### 1.2 Is your (deep) network better than SVM? Why or why not? (4 points)

我的network比SVM表現來的更好，可能原因為
1. SVM的input只選用了X裡面前兩個feature, 在我的模型中我用了全部Ｘ裡面的feature(4個)
2. 我將batch size調大使得每次更新weight時所用來算loss的sample數量變為30
3. 將epoch調大，使model可以訓練更多次才做預測
4. 將層數加深並加入dropout避免overfit，且中間的activation func都用relu只保留正相關資訊
5. 因為是multiclass classfication , output層的activation func改成使用softmax
6. 使用adam作為optimizer
